<a href="https://colab.research.google.com/github/kobars/Assignment-05-W05-First-Machine-Learning-Project/blob/master/Assignment_05_W05_First_Machine_Learning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this colabs, we try to do a binary classification to determine whether a video published on YouTube will reach a certain number of views. The dataset is taken from Kaggle which can be seen from the following link https://www.kaggle.com/datasnaek/youtube-new

# Use the right version of TensorFlow

The following hidden code cell ensures that the Colab will run on TensorFlow 2.X.

In [0]:
#@title Run on TensorFlow 2.x
%tensorflow_version 2.x

# Call the import statements

The following code imports the necessary modules.

In [2]:
#@title Load the imports

# from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import tensorflow as tf
import string,math,pickle
from textblob import TextBlob
from tensorflow import feature_column
from tensorflow.keras import layers
from matplotlib import pyplot as plt

# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
# tf.keras.backend.set_floatx('float32')

print("Ran the import statements.")

Ran the import statements.


In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/kobars/Assignment-05-W05-First-Machine-Learning-Project/master/Ind-daily-updated-data/merged.csv')

In [4]:
df

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,MF1queeLUU8,AISYAH ISTRI RASULULLAH - SYAKIR DAULAY [ COV...,2020-03-29T10:00:05.000Z,UC2zvYA-I_HtKfQnjNGY2VFA,Syakir Daulay,10,20.01.04,di rumah aja|aisyah istri rasulullah|aisyah|sy...,6103246,408534,4498,27080,https://i.ytimg.com/vi/MF1queeLUU8/default.jpg,False,False,Assalamualaikum wr. wb.Alhamdulillah hari ini ...
1,aN0ZnoRg_IY,SABYAN - AISYAH ISTRI RASULULLAH | COVER,2020-03-28T02:10:45.000Z,UCeZA77FZSG4YzBcvBJ9eynA,SABYAN,10,20.01.04,Nissa sabyan|sabyan gambus|sabyan aisyah|ya al...,7830616,553305,7770,57050,https://i.ytimg.com/vi/aN0ZnoRg_IY/default.jpg,False,False,Assalamualaikum wr. wb.Alhamdulillah hari ini ...
2,ZphLn5AJH2Y,Amerika Serikat Kini Jadi Episentrum Corona,2020-03-30T05:30:09.000Z,UCGN9JsnkvK05v2lnTI_-uGA,CNBC Indonesia,22,20.01.04,cnbc indonesia|ekonomi|bisnis|indonesia|corona...,1208631,8065,776,2963,https://i.ytimg.com/vi/ZphLn5AJH2Y/default.jpg,False,False,Kasus positif virus Corona di Amerika Serikat ...
3,ST28wZWevMI,AISYAH ISTRI RASULULLAH - COVEER BY TRI SUAKA,2020-03-30T12:00:22.000Z,UCUE-BF1Q5E6Sz1KidSnSRYw,musisi jogja project,10,20.01.04,Cover musik|musisi jalanan|musisi jogja|pengam...,1459470,73587,1269,3602,https://i.ytimg.com/vi/ST28wZWevMI/default.jpg,False,False,AISYAH ISTRI RASULULLAHOriginal Song by Projec...
4,lTnjVMV9jmY,KENAPA CUMA LOE YG BERANI NGOMONG?! JAKARTA SU...,2020-03-28T03:57:47.000Z,UCYk4LJI0Pr6RBDWowMm-KUw,Deddy Corbuzier,24,20.01.04,anies baswedan|jokowi|Covid19|Indonesia|jakart...,4953538,223743,10107,56781,https://i.ytimg.com/vi/lTnjVMV9jmY/default.jpg,False,False,#indonesia #corona #covid19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3402,7dye45y2g8I,AISYAH ISTERI ROSULULLOH | VEVE ZULFIKAR BASYA...,2020-03-26T11:07:37.000Z,UCKxHsgyk3VLq_5DL63RX2zQ,VEVE ZULFIKAR,10,20.31.03,[none],194236,11250,147,504,https://i.ytimg.com/vi/7dye45y2g8I/default.jpg,False,False,terima kasih banyak buat temen2 yg usul cover ...
3403,X9s_PXxT2zg,Sedang Sayang Sayangnya - Mawar De Jongh | Cov...,2020-03-24T10:00:07.000Z,UC9hx5-f_s3L-mw_7ORsVrLg,Billy Joe Ava,10,20.31.03,billy joe ava|billy|cover lagu|cover|cover bil...,196234,11826,56,772,https://i.ytimg.com/vi/X9s_PXxT2zg/default.jpg,False,False,Sedang Sayang Sayangnya - Mawar De Jongh Cover...
3404,vx8BZpK5l0s,TEMBAK PANTAT NYA!! (RADITYA DIKA DAN BAYI M...,2020-03-27T04:54:32.000Z,UCYk4LJI0Pr6RBDWowMm-KUw,Deddy Corbuzier,24,20.31.03,raditya dika|gabut|bayi manusia|stand up|lucu|...,1521733,62635,787,6442,https://i.ytimg.com/vi/vx8BZpK5l0s/default.jpg,False,False,#radityadika #santuy #gabut
3405,JX_Q8h4duaM,Air Suspension Drifting di Rumah! | Physical D...,2020-03-27T09:00:08.000Z,UCmA-OmY_ZW7VZkET-64Psdg,Garasi Drift,2,20.31.03,[none],413096,18671,293,1376,https://i.ytimg.com/vi/JX_Q8h4duaM/default.jpg,False,False,English Subtitle Available---------\rGarasi Dr...


In [5]:
# is there a missing value?
df.isna().sum()

video_id              0
title                 0
publishedAt           0
channelId             0
channelTitle          0
                     ..
comment_count         0
thumbnail_link        0
comments_disabled     0
ratings_disabled      0
description          76
Length: 16, dtype: int64

In [6]:
# Displaying column info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3407 entries, 0 to 3406
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_id           3407 non-null   object
 1   title              3407 non-null   object
 2   publishedAt        3407 non-null   object
 3   channelId          3407 non-null   object
 4   channelTitle       3407 non-null   object
 5   categoryId         3407 non-null   int64 
 6   trending_date      3407 non-null   object
 7   tags               3407 non-null   object
 8   view_count         3407 non-null   int64 
 9   likes              3407 non-null   int64 
 10  dislikes           3407 non-null   int64 
 11  comment_count      3407 non-null   int64 
 12  thumbnail_link     3407 non-null   object
 13  comments_disabled  3407 non-null   bool  
 14  ratings_disabled   3407 non-null   bool  
 15  description        3331 non-null   object
dtypes: bool(2), int64(5), object(9)
memory usa

In [7]:
df.shape

(3407, 16)

In [0]:
df['trending_date'] = pd.to_datetime(df['trending_date'],format='%y.%d.%m')

In [9]:
df['trending_date'].sort_values

<bound method Series.sort_values of 0      2020-04-01
1      2020-04-01
2      2020-04-01
3      2020-04-01
4      2020-04-01
          ...    
3402   2020-03-31
3403   2020-03-31
3404   2020-03-31
3405   2020-03-31
3406   2020-03-31
Name: trending_date, Length: 3407, dtype: datetime64[ns]>

# Feature Extraction

The dataset does not have labels for how long a video stays in the trending list. Therefore, the data is reformatted for the purpose of this project before preforming data extraction

In [0]:
def remove_punctuation (doc):
    return " ".join("".join([" " if ch in string.punctuation else ch for ch in doc]).split())

def tfidf_fun(word, blob, bloblist):
    tf=blob.words.count(word) / len(blob.words)
    n_containing=sum(1 for blob2 in bloblist if word in blob2.words)
    idf=math.log(len(bloblist) / (1 + n_containing))
    return tf * idf

def avg_tfidf_fun(blob, bloblist):
    avg_tfidf=0
    for wordI in blob.words:
        avg_tfidf=avg_tfidf+tfidf_fun(wordI, blob, bloblist)
    if blob.words.__len__() != 0:
        avg_tfidf=avg_tfidf/blob.words.__len__()
        return avg_tfidf
    else:
        return 0

In [0]:
numFeatures = 25

In [0]:
#create the features and labels dataframe
df_features = pd.DataFrame(np.zeros([df.shape[0],numFeatures]),columns=['days_was_trending', 'title_polarity', 'title_subjectivity','title_tfidf' ,
                                                                             'tags_polarity', 'tags_subjectivity' , 'tags_tfidf','description_polarity',
                                                                             'description_subjectivity', 'description_tfidf',
                                                                             'categoryId','days_since_uploaded','view_count',
                                                                             'likes','dislikes','comment_count','like_rate','dislike_rate',
                                                                             'comment_rate','view_change','like_change','dislike_change','comment_change',
                                                                             'comments_disabled','ratings_disabled'])

In [0]:
df_label = pd.DataFrame(np.zeros([df.shape[0],2]),columns=['video_ind','days_will_be_trending'])

In [14]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [0]:
unique_ids=df['video_id'].unique()
bloblist=[]
for docI in range(unique_ids.shape[0]):
    vedI_data = df[df['video_id'] == unique_ids[docI]]
    vedI_data=vedI_data.reset_index(drop=True)
    document = vedI_data.title[0]+ " " +vedI_data.tags[0]+" " +str(vedI_data.description[0])
    bloblist.append(TextBlob(remove_punctuation(document)))

In [16]:
count1=0
for vidI in range(100):#range(unique_ids.shape[0]): #TODO remove the comment to use all the videos and not just 10
    print('Extracting the features of video #:' + str(vidI))
    vedI_data=df[df['video_id']==unique_ids[vidI]]
    vedI_data=vedI_data.reset_index(drop=True)
    total_num_trending=vedI_data.shape[0]

    titleI = TextBlob(remove_punctuation(vedI_data.title[0])) #removing the punctuation results in changing sentiment results slightly
    tagsI = TextBlob(remove_punctuation(vedI_data.tags[0]))
    descriptionI = TextBlob(remove_punctuation(str(vedI_data.description[0])))
    title_tfidf=avg_tfidf_fun(titleI,bloblist)
    tags_tfidf = avg_tfidf_fun(tagsI, bloblist)
    description_tfidf = avg_tfidf_fun(descriptionI, bloblist)

    trending_date=pd.to_datetime(vedI_data.trending_date[0], format='%y.%d.%m')
    published_date=pd.to_datetime(vedI_data.publishedAt[0], format='%Y-%m-%dT%H:%M:%S.%fZ')
    date_published_trending=trending_date -published_date
    days_published_trending=float(date_published_trending._d)+(float(date_published_trending._h)/24)

    for dayI in range(total_num_trending):
        #print('day number :' + str(dayI))
        df_label.video_ind[count1] = vidI
        df_label.days_will_be_trending[count1]=total_num_trending-dayI-1

        df_features.days_was_trending[count1]=dayI+1

        df_features.title_polarity[count1] =titleI.sentiment.polarity
        df_features.title_subjectivity[count1] = titleI.sentiment.subjectivity
        df_features.title_tfidf[count1]=title_tfidf

        df_features.tags_polarity[count1] =tagsI.sentiment.polarity
        df_features.tags_subjectivity[count1] = tagsI.sentiment.subjectivity
        df_features.tags_tfidf[count1]=tags_tfidf

        df_features.description_polarity[count1] =descriptionI.sentiment.polarity #average for all the sentences
        df_features.description_subjectivity[count1] = descriptionI.sentiment.subjectivity
        df_features.description_tfidf[count1]=description_tfidf

        df_features.categoryId[count1]=vedI_data.categoryId[dayI]
        df_features.days_since_uploaded[count1]=days_published_trending+dayI+1
        df_features.view_count[count1]=vedI_data.view_count[dayI]
        df_features.likes[count1]=vedI_data.likes[dayI]
        df_features.dislikes[count1]=vedI_data.dislikes[dayI]
        df_features.comment_count[count1]=vedI_data.comment_count[dayI]

        df_features.like_rate[count1]=vedI_data.likes[dayI]/vedI_data.view_count[dayI]
        df_features.dislike_rate[count1]=vedI_data.dislikes[dayI]/vedI_data.view_count[dayI]
        df_features.comment_rate[count1]=vedI_data.comment_count[dayI]/vedI_data.view_count[dayI]

        if dayI==0: #no change for day 1 but give it 1 for each feature ;)
            df_features.view_change[count1]=1
            df_features.like_change[count1]=1
            df_features.dislike_change[count1]=1
            df_features.comment_change[count1]=1
        else:
            df_features.view_change[count1]=vedI_data.view_count[dayI]-vedI_data.view_count[dayI-1]
            df_features.like_change[count1]=vedI_data.likes[dayI]-vedI_data.likes[dayI-1]
            df_features.dislike_change[count1]=vedI_data.dislikes[dayI]-vedI_data.dislikes[dayI-1]
            df_features.comment_change[count1]=vedI_data.comment_count[dayI]-vedI_data.comment_count[dayI-1]

        df_features.comments_disabled[count1] = np.double(vedI_data.comments_disabled[dayI])
        df_features.ratings_disabled[count1] = np.double(vedI_data.ratings_disabled[dayI])
        count1+=1

with open('/NLP_data.pkl', 'wb') as Data:
    pickle.dump([df_features, df_label, numFeatures, bloblist, unique_ids], Data)

Extracting the features of video #:0
Extracting the features of video #:1
Extracting the features of video #:2
Extracting the features of video #:3
Extracting the features of video #:4
Extracting the features of video #:5
Extracting the features of video #:6
Extracting the features of video #:7
Extracting the features of video #:8
Extracting the features of video #:9
Extracting the features of video #:10
Extracting the features of video #:11
Extracting the features of video #:12
Extracting the features of video #:13
Extracting the features of video #:14
Extracting the features of video #:15
Extracting the features of video #:16
Extracting the features of video #:17
Extracting the features of video #:18
Extracting the features of video #:19
Extracting the features of video #:20
Extracting the features of video #:21
Extracting the features of video #:22
Extracting the features of video #:23
Extracting the features of video #:24
Extracting the features of video #:25
Extracting the feature

In [17]:
df_features

,days_was_trending,title_polarity,title_subjectivity,title_tfidf,tags_polarity,tags_subjectivity,tags_tfidf,description_polarity,description_subjectivity,description_tfidf,categoryId,days_since_uploaded,view_count,likes,dislikes,comment_count,like_rate,dislike_rate,comment_rate,view_change,like_change,dislike_change,comment_change,comments_disabled,ratings_disabled
0,1.0,0.0,0.0,0.6,0.0,0.0,0.4,0.2,0.5,0.0,10.0,1.0,6103246.0,408534.0,4498.0,27080.0,0.1,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
1,2.0,0.0,0.0,0.6,0.0,0.0,0.4,0.2,0.5,0.0,10.0,2.0,10405554.0,575991.0,7357.0,35331.0,0.1,0.0,0.0,4302308.0,167457.0,2859.0,8251.0,0.0,0.0
2,3.0,0.0,0.0,0.6,0.0,0.0,0.4,0.2,0.5,0.0,10.0,3.0,13437136.0,654742.0,9338.0,39023.0,0.0,0.0,0.0,3031582.0,78751.0,1981.0,3692.0,0.0,0.0
3,4.0,0.0,0.0,0.6,0.0,0.0,0.4,0.2,0.5,0.0,10.0,4.0,16263223.0,738446.0,11755.0,43520.0,0.0,0.0,0.0,2826087.0,83704.0,2417.0,4497.0,0.0,0.0
4,5.0,0.0,0.0,0.6,0.0,0.0,0.4,0.2,0.5,0.0,10.0,5.0,23042934.0,933351.0,19470.0,55345.0,0.0,0.0,0.0,6779711.0,194905.0,7715.0,11825.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df_label

,video_ind,days_will_be_trending
0,0.0,9.0
1,0.0,8.0
2,0.0,7.0
3,0.0,6.0
4,0.0,5.0
...,...,...
3402,0.0,0.0
3403,0.0,0.0
3404,0.0,0.0
3405,0.0,0.0


In [21]:
from __future__ import print_function, division
import numpy as np, pickle, os
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy import stats
from sklearn.model_selection import train_test_split
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from xgboost import plot_importance

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [28]:
# reformat the data
list_videos = np.array(df_label['video_ind'])  # contains indeces for all the videos with repetition.
print("list videos ", list_videos)
num_videos = np.int32(list_videos[-1])
print("num videos ", num_videos)
list_features = np.empty((num_videos,), dtype=object)
print("list features ", list_features)
list_label = np.empty((num_videos,), dtype=object)
print("list label ", list_label)
for i in range(num_videos):
    list_features[i] = df_features[df_label['video_ind'] == i]
    print("list_features[i] ", list_features[i])
    list_label[i] = df_label.loc[df_label['video_ind'] == i, 'days_will_be_trending']
    print("list_label[i] ", list_label[i])

list videos  [0. 0. 0. ... 0. 0. 0.]
num videos  0
list features  []
list label  []


In [0]:
# Splitting the data into (60% training + 20% validation), 20%testing
list_features_train, list_features_test, list_label_train, list_label_test = train_test_split(df_features,
                                                                                              df_label,
                                                                                              test_size=0.2,
                                                                                              random_state=1)

In [0]:
list_features_train, list_features_val, list_label_train, list_label_val = train_test_split(list_features_train,
                                                                                                    list_label_train,
                                                                                                    test_size=0.2,
                                                                                                    random_state=1)

In [0]:
num_videos_train = list_features_train.shape[0]  # number of videos for training
num_videos_val = list_features_val.shape[0]  # number of videos for validation
num_videos_test = list_features_test.shape[0]  # number of videos for testing

In [38]:
num_videos_train

2180

In [42]:
num_videos_test

682

In [40]:
num_videos_val

545

In [0]:
########################### Plotting functions and others #############################################################
def plot_fun(loss_train, loss_val, accuracy_train_All, accuracy_val_All):
    plt.cla()
    plt.subplot(1, 2, 1)
    plt.plot(np.arange(1, loss_train.__len__() + 1), loss_train, 'b', np.arange(1, loss_val.__len__() + 1),
             loss_val, 'g')
    plt.ylabel('Loss')
    plt.xlabel('Epoch number')
    plt.legend(['Training Loss', 'Validation Loss'])
    # plt.ylim([0, 3])

    plt.subplot(1, 2, 2)
    plt.plot(np.arange(1, accuracy_train_All.__len__() + 1), accuracy_train_All, 'b-s',
             np.arange(1, accuracy_train_All.__len__() + 1),
             accuracy_val_All, 'g-^')
    plt.ylabel('corrcoef')
    plt.xlabel('Epoch number')
    plt.legend(['Training corrcoef', 'Validation corrcoef'])
    # plt.ylim([0,100])
    plt.draw()
    plt.pause(0.000000001)

In [0]:
def plot_dis(x):
    num_bins = range(int(min(x)),int(max(x)+2))
    # the histogram of the data
    n, bins, patches = plt.hist(x, num_bins, normed=1, facecolor='blue', alpha=0.5)

    # add a 'best fit' line
    #y = mlab.normpdf(bins, np.mean(x), np.std(x))
    #plt.plot(bins, y, 'r--')
    plt.xlabel('Days')
    plt.ylabel('Probability')
    plt.title(r'Training Data Distribution')

    # Tweak spacing to prevent clipping of ylabel
    plt.subplots_adjust(left=0.15)
    plt.show()

def plot_feature_imp(model):
    plot_importance(model)

In [29]:
################################### XGBoost hyperparameters #################################################

SaveDirParent = '/'
num_classes = 1
learning_rateI=0.1  #step size shrinkage used to prevent overfitting. Range is [0,1]
n_estimatorsRange=[30]#range(10,100,20) #[10,30,50,..]  number of trees you want to build.
max_depthRange=[5]#range(3,10,2) # determines how deeply each tree is allowed to grow during any boosting round.
colsample_byteeRange=[0.3]#[i/10.0 for i in range(1,5)] #percentage of features used per tree. High value can lead to overfitting.
#subsampleI=0.8 #percentage of samples used per tree. Low value can lead to underfitting.
gamaI=[0.3]#[i/10.0 for i in range(0,6)] #controls whether a given node will split based on the expected reduction
                                        # in loss after the split. A higher value leads to fewer splits. Supported only for tree-based learners.
#reg_alphaRange=[1e-5, 1e-2, 0.1, 1, 100] #alpha: L1 regularization on leaf weights. A large value leads to more regularization.

#Example https://www.datacamp.com/community/tutorials/xgboost-in-python#what
#Tunning https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/


num_classes = 1
# truncated_backprop_length = ? # not used
RandomStart = 1 / 4  # pick a random start for each signal between 0 and (RandomStart*signal length)

################################ Import the dataset #########################################
exists = os.path.isfile('/NLP_data_reformed.pkl')
if exists:
    with open('/NLP_data_reformed.pkl', 'rb') as Data:
        [array_features_train, array_features_val, array_features_test, array_label_train,
         array_label_val,
         array_label_test, num_samples_train, num_samples_val, num_samples_test, features_mean, features_std,
         numFeatures, bloblist, unique_ids]= pickle.load(Data)
else: # Load and reformat the split data data
    exists = os.path.isfile('/NLP_data_splits.pkl')
    if exists:
        with open('/NLP_data_splits.pkl', 'rb') as Data:
            [list_features_train, list_features_val, list_features_test, list_features_train, list_label_train,
             list_label_val,
             list_label_test, num_videos_train, num_videos_val, num_videos_test, features_mean, features_std,
             numFeatures,
             bloblist, unique_ids] = pickle.load(Data)
    else:  # Load and split the original data
        with open('/NLP_data.pkl', 'rb') as Data:
            [df_features, df_label, numFeatures, bloblist, unique_ids] = pickle.load(Data)

        # reformat the data
        list_videos = np.array(df_label['video_ind'])  # contains indeces for all the videos with repetition.
        num_videos = np.int32(list_videos[-1])
        list_features = np.empty((num_videos,), dtype=object)
        list_label = np.empty((num_videos,), dtype=object)
        for i in range(num_videos):
            list_features[i] = df_features[df_label['video_ind'] == i]
            list_label[i] = df_label.loc[df_label['video_ind'] == i, 'days_will_be_trending']

        # Splitting the data into (60% training + 20% validation), 20%testing
        list_features_train, list_features_test, list_label_train, list_label_test = train_test_split(list_features,
                                                                                                      list_label,
                                                                                                      test_size=0.2,
                                                                                                      random_state=1)

        list_features_train, list_features_val, list_label_train, list_label_val = train_test_split(list_features_train,
                                                                                                    list_label_train,
                                                                                                    test_size=0.2,
                                                                                                    random_state=1)

        num_videos_train = list_features_train.shape[0]  # number of videos for training
        num_videos_val = list_features_val.shape[0]  # number of videos for validation
        num_videos_test = list_features_val.shape[0]  # number of videos for testing

        # normalizing the features based on mean and std of training data
        num_samples = 0
        features_mean = np.zeros(numFeatures)
        features_std = np.zeros(numFeatures)
        for video_i in range(num_videos_train):
            features_mean = np.sum(list_features_train[video_i], axis=0) + features_mean
            features_std = np.std(list_features_train[video_i], axis=0) + features_std
            num_samples = num_samples + list_features_train[video_i].shape[0]

        features_mean = features_mean / num_samples
        features_std = features_std / num_samples

        # normalize training data
        for video_i in range(num_videos_train):
            list_features_train[video_i] = (list_features_train[video_i] - features_mean) / features_std

        # normalize validation data
        for video_i in range(num_videos_val):
            list_features_val[video_i] = (list_features_val[video_i] - features_mean) / features_std
        # normalize testing data
        for video_i in range(num_videos_test):
            list_features_test[video_i] = (list_features_test[video_i] - features_mean) / features_std

        with open('/NLP_data_splits.pkl', 'wb') as Data:
            pickle.dump(
                [list_features_train, list_features_val, list_features_test, list_features_train, list_label_train,
                 list_label_val,
                 list_label_test, num_videos_train, num_videos_val, num_videos_test, features_mean, features_std,
                 numFeatures,
                 bloblist, unique_ids], Data)

    # reformat the data to one array for training , validation testing
    num_samples_train = 0
    for video_i in range(num_videos_train):
        num_samples_train = num_samples_train + list_features_train[video_i].shape[0]
    num_samples_val = 0
    for video_i in range(num_videos_val):
        num_samples_val = num_samples_val + list_features_val[video_i].shape[0]
    num_samples_test = 0
    for video_i in range(num_videos_test):
        num_samples_test = num_samples_test + list_features_test[video_i].shape[0]

    array_features_train = np.zeros([num_samples_train, numFeatures])
    array_label_train = np.zeros([num_samples_train])
    countI = 0
    for video_i in range(num_videos_train):
        indFrom = countI
        indTo = indFrom + list_features_train[video_i].shape[0]
        array_features_train[indFrom:indTo, :] = np.array(list_features_train[video_i])
        array_label_train[indFrom:indTo] = np.array(list_label_train[video_i])
        countI = indTo

    array_features_val = np.zeros([num_samples_val, numFeatures])
    array_label_val = np.zeros([num_samples_val])
    countI = 0
    for video_i in range(num_videos_val):
        indFrom = countI
        indTo = indFrom + list_features_val[video_i].shape[0]
        array_features_val[indFrom:indTo, :] = np.array(list_features_val[video_i])
        array_label_val[indFrom:indTo] = np.array(list_label_val[video_i])
        countI = indTo

    array_features_test = np.zeros([num_samples_test, numFeatures])
    array_label_test = np.zeros([num_samples_test])
    countI = 0
    for video_i in range(num_videos_test):
        indFrom = countI
        indTo = indFrom + list_features_test[video_i].shape[0]
        array_features_test[indFrom:indTo, :] = np.array(list_features_test[video_i])
        array_label_test[indFrom:indTo] = np.array(list_label_test[video_i])
        countI = indTo

    with open('/NLP_data_reformed.pkl', 'wb') as Data:
        pickle.dump(
            [array_features_train, array_features_val, array_features_test, array_label_train,
             array_label_val,
             array_label_test, num_samples_train, num_samples_val, num_samples_test, features_mean, features_std,
             numFeatures, bloblist, unique_ids], Data)

#Random oversampling
ros = SMOTE(random_state=42,k_neighbors=3)
array_features_train, array_label_train = ros.fit_resample(array_features_train, array_label_train)


# Saving results parameters
savingTime = 1  # save after N epochs
Results_train_All = np.zeros((len(n_estimatorsRange) * len(max_depthRange) * len(colsample_byteeRange),
                              5))  # saving the results for each fold in CV, for 160 trials and for 5 metrics
Results_val_All = np.zeros((len(n_estimatorsRange) * len(max_depthRange) * len(colsample_byteeRange), 5))  #
CVpredictionsMaxCorr = np.zeros((num_samples_test))
CVpredictionsMinRMSE = np.zeros((num_samples_test))
countSave = 0
highestInnCorr = -1  # initial values
lowestInnRMSE = 100

for n_estimatorsI in  n_estimatorsRange:
    for max_depthI in max_depthRange:
        for colsample_byteeI in colsample_byteeRange:
            xg_reg = xgb.XGBRegressor(objective='reg:linear', colsample_bytree=colsample_byteeI,
                                      learning_rate=learning_rateI,
                                      max_depth=max_depthI, gama=gamaI, n_estimators=n_estimatorsI)
            xg_reg.fit(array_features_train, array_label_train)

            # Results on training data
            predictionsPerDay = xg_reg.predict(array_features_train)

            Results_train_All[countSave, 0] = np.sqrt(mean_squared_error(array_label_train, predictionsPerDay))
            Results_train_All[countSave, 1] = mean_absolute_error(array_label_train, predictionsPerDay)
            Results_train_All[countSave, 2] = r2_score(array_label_train, predictionsPerDay)
            corrI = stats.pearsonr(array_label_train, predictionsPerDay)
            Results_train_All[countSave, 3] = corrI[0]
            Results_train_All[countSave, 4] = corrI[1]

            # Results on validation data
            predictionsPerDay = xg_reg.predict(array_features_val)

            Results_val_All[countSave, 0] = np.sqrt(mean_squared_error(array_label_val, predictionsPerDay))
            Results_val_All[countSave, 1] = mean_absolute_error(array_label_val, predictionsPerDay)
            Results_val_All[countSave, 2] = r2_score(array_label_val, predictionsPerDay)
            corrI = stats.pearsonr(array_label_val, predictionsPerDay)
            Results_val_All[countSave, 3] = corrI[0]
            Results_val_All[countSave, 4] = corrI[1]
            print("XGboost with estimators # %d, max depth # %d, training RMSE %.2f, validation RMSE %.2f" % (
            n_estimatorsI, max_depthI,
            Results_train_All[
                countSave, 0],
            Results_val_All[
                countSave, 0]))

            # Saving the model with the highest validation correlation

            if (Results_val_All[countSave, 3] > highestInnCorr and Results_val_All[countSave, 3] ==
                    Results_val_All[countSave, 3]):
                save_path = SaveDirParent + 'xgbMode' + '_CorrCV.dat'
                pickle.dump(xg_reg, open(save_path, "wb"))
                highestInnCorr = Results_val_All[countSave, 3]
                print("Model saved in file: %s" % save_path)
            if (Results_val_All[countSave, 0] < lowestInnRMSE and Results_val_All[countSave, 0] ==
                    Results_val_All[countSave, 0]):
                save_path = SaveDirParent + 'xgbMode' + '_RMSECV.dat'
                pickle.dump(xg_reg, open(save_path, "wb"))
                lowestInnRMSE = Results_val_All[countSave, 0]
                print("Model saved in file: %s" % save_path)

            countSave = countSave + 1




# Results on Testing data
# Using saved best model with highest correlation to find testing predictions and save it
save_path = SaveDirParent +'xgbMode'+ '_CorrCV.dat'
xg_reg = pickle.load(open(save_path, "rb"))
predictions_MaxCorr= xg_reg.predict(array_features_test)

# Using saved best model with lowest RMSE to find testing predictions and save it
save_path = SaveDirParent + 'xgbMode' + '_RMSECV.dat'
xg_reg = pickle.load(open(save_path, "rb"))
predictions_MinRMSE = xg_reg.predict(array_features_test)

# Finding Final Testing Results
corrI = stats.pearsonr(array_label_test, predictions_MaxCorr)
class_reportMaxCorr = [np.sqrt(mean_squared_error(array_label_test, predictions_MaxCorr)),
                       mean_absolute_error(array_label_test, predictions_MaxCorr),
                       r2_score(array_label_test, predictions_MaxCorr), corrI[0], corrI[1]]
print(
    "Final testing using Max validation Corr: RMSE %.2f, MAE %.2f, R2 score %.2f, Correlation coefficient %.2f (p=%.4f)." % (
        np.sqrt(mean_squared_error(array_label_test, predictions_MaxCorr)),
        mean_absolute_error(array_label_test, predictions_MaxCorr),
        r2_score(array_label_test, predictions_MaxCorr),
        corrI[0], corrI[1]))
corrI = stats.pearsonr(array_label_test, predictions_MinRMSE)
class_reportMinRMSE = [np.sqrt(mean_squared_error(array_label_test, predictions_MinRMSE)),
                       mean_absolute_error(array_label_test, predictions_MinRMSE),
                       r2_score(array_label_test, predictions_MinRMSE), corrI[0], corrI[1]]

print(
    "Final testing using Min validation RMSE: RMSE %.2f, MAE %.2f, R2 score %.2f, Correlation coefficient %.2f (p=%.4f)." % (
        np.sqrt(mean_squared_error(array_label_test, predictions_MinRMSE)),
        mean_absolute_error(array_label_test, predictions_MinRMSE),
        r2_score(array_label_test, predictions_MinRMSE),
        corrI[0], corrI[1]))

###############Saving the variables#################
# Saving the objects:
with open(SaveDirParent + 'XGboostResults.pkl', 'wb') as Results:  # Python 3: open(..., 'wb')
    pickle.dump(
        [Results_train_All, Results_val_All, class_reportMaxCorr, class_reportMinRMSE,
         predictions_MinRMSE, array_label_test, predictions_MaxCorr], Results)

ValueError: ignored

In [25]:
list_features

array([], dtype=object)

In [26]:
list_features_train

NameError: ignored